### import
- 다운자료의 날짜는 전일(기준일)로 한다!
- 계좌별진행상황 : 기준일 다음날 아침에 다운받기, 틀린 계좌수정하기, 통합하기, 전산 올리기, 그 결과를 엑셀로 저장하기(기준일까지의 진행상황 반영하기)
  - t+1에 한번만 받으면 됨
  - 현재상황만 실시간이고, 나머지는 전주까지의 기록
  - 따라서 기준일이 금요일인 경우 다음주 월요일(t+1)에 다운 받으면 기준데이터에 입금내역 반영하지 않아도 됨.(전산 opb는 자산정리 통합파일에서 수정)
  - 기준일이 목요일인 경우 금요일(t+1)에 받은 계좌별진행상황에 월~목요일 상환내역을 반영해야함
- 상환내역 : 결과를 엑셀로 저장하기(기준일까지의 진행상황 반영하기)
  - 기준일이 금,토,일요일인 경우, 상환내역을 고려하지 않아도 된다.
  - 기준일이 금,토,일요일이 아닌 경우, 기준일이 속한 주의 월요일 ~ t일 상환내역을 모두 읽어야
  - 여러회차를 한꺼번에 상환할 수 있으니, 마지막 회차, 전체입금원금,이자,기타비용을 계산해야.

In [33]:
import os, re, pickle
import pandas as pd
import numpy as np
from os.path import join
from tqdm import tqdm

# dtype 정의 (사이트 파일은 읽지 말자. 할 게 너무 많음)
# 나우리 타입 반영
nauri_dtype = {"채무자키":str, "계좌키":str, '보증인키':str, '신용회복키' : str, "계좌번호":str, "심의차수":str, "변제금수취계좌":str}
# # 신복 사이트 타입 반영
# nauri_dtype = {'채무자키' : str, '계좌키' : str, '보증인키':str, '신용회복키' : str, '심의차수' : str, '조정전이율' : str, '조정후이율' : str, '총상환기간' : str, '유예기간' : str, '원금균등상환기간' : str, '원리균등상환기간' : str, '이자상환기간' : str, '납입회차' : str, '연체기간' : str, '원금균등시작회차' : str, '원금균등종료회차' : str, '원리균등시작회차' : str, '원리균등종료회차' : str, '이자상환시작회차' : str, '이자상환종료회차' : str, '감면율' : str, '주채무자주민번호' : str, '시작회차1' : str, '종료회차1' : str, '채무조정1적용이율1' : str, '채무조정2적용이율1' : str, '시작회차2' : str, '종료회차2' : str, '채무조정1적용이율2' : str, '채무조정2적용이율2' : str, '시작회차3' : str, '종료회차3' : str, '채무조정1적용이율3' : str, '채무조정2적용이율3' : str, '시작회차4' : str, '종료회차4' : str, '채무조정1적용이율4' : str, '채무조정2적용이율4' : str, '시작회차5' : str, '종료회차5' : str, '채무조정1적용이율5' : str, '채무조정2적용이율5' : str, '시작회차6' : str, '종료회차6' : str, '채무조정1적용이율6' : str, '채무조정2적용이율6' : str, '시작회차7' : str, '종료회차7' : str, '채무조정1적용이율7' : str, '채무조정2적용이율7' : str, '시작회차8' : str, '종료회차8' : str, '채무조정1적용이율8' : str, '채무조정2적용이율8' : str, '시작회차9' : str, '종료회차9' : str, '채무조정1적용이율9' : str, '채무조정2적용이율9' : str, '시작회차10' : str, '종료회차10' : str, '채무조정1적용이율10' : str, '채무조정2적용이율10' : str, '재조정횟수' : str, '수정조정횟수' : str, '일시납감면율' : str, '거치기간' : str, '체증전구간의시작회차' : str, '체증전구간의종료회차' : str, '체증전구간의기준기간' : str, '변제금수취계좌' : str, '담보권실행유예기간':str
# }
deposit_dtype = {'채무자키':str, '계좌키':str, '분납키':str, '담당자키':str, '일시납감면율':str, '계좌번호':str}

### 파일 읽기

In [34]:
# 계좌별 진행상황 파일 읽기
path_base = r"D:\3.자산\신용회복\신용회복 전체데이터\계좌별 진행상황"
########################################################
basedate = "231010"
company = "솔림"
########################################################
sufix = "_업로드결과"

# 읽을 파일명
filename_read = basedate + "_계좌별 진행상황 조회_" + company + sufix
fullpath_read = join(path_base, filename_read+".xlsx")

# 계좌별 진행상황 조회 파일 읽기
raw_ori = pd.read_excel(fullpath_read, dtype=nauri_dtype).fillna("")

In [35]:
# 재시작 위해 복사본 만들기
raw = raw_ori.copy()

# 불필요한 열 삭제
raw.drop(columns=["채무자명","채무상태", "현재채권구분","등록파일","수정파일", "삭제여부","등록일자","등록유저","수정일자","수정유저","확인결과","저장결과"], axis=1, inplace=True)

# 새로운 열을 추가 : insert 여러번 쓰니 성능 경고나와서
진행구분 = pd.Series([None] * len(raw), name='진행구분')
반영OPB = pd.Series([None] * len(raw), name='반영OPB')
상환구분 = pd.Series([None] * len(raw), name='상환구분')

position = 4
raw = pd.concat([raw.iloc[:, :position], 진행구분, 반영OPB, 상환구분, raw.iloc[:, position:]], axis=1)

# 전산미등록건 처리 (강민경)
미등록주민번호 = ['771027-2221231']
cond = raw['신청인주민번호'].isin(미등록주민번호)
raw.loc[cond, '채무자키'] = '전산미등록건'
raw.loc[cond, '계좌키'] = '전산미등록건'

### 진행구분, 반영OPB, 상환구분 작성

In [36]:
# 진행구분열 작성-------------------------------
x = raw.신청인진행상태
y = raw.계좌진행상태내용
z = raw.계좌번호

conds = [
    (x == '정상이행') & (y == '채무조정포함'),
    (y == "채무조정제외"),
    (x.str.contains(r"완제|상환") | y.str.contains(r"완제|상환") | z.str.contains(r"완제|상환")),
    (z == "채권신고전"),
    (x.str.contains(r"심사반송|중도실효|합의서체결포기")),
]
values = ["확정", "채무조정제외","완제","채권신고전","실효"]
default_value = "미확정"

raw["진행구분"] = np.select(conds, values, default_value)

# 반영OPB - 확정미확정 건-------------------------------
x = raw.진행구분
# 확정시 우선순위
a = raw.상환후잔액
b = raw.원금균등채무액
c= raw.조정후원금
d = raw.조정전원금

conds = [(x == "미확정"), (x =="확정")&(a!=0), b!=0, c!=0, d!=0]

# 조건에 따라 값을 설정
values = [d, a, b, c, d]

# 기본값 설정
default_value = np.nan

# 반영OPB 열 설정
raw["반영OPB"] = np.select(conds, values, default_value)


# 상환구분 작성 - 확정미확정 건-------------------------------
x = raw.감면방식

conds = [x=="원금균등", x==""]

# 조건에 따라 값을 설정
values = ["프리", ""]

# 기본값 설정
default_value = "개인"

# 반영OPB 열 설정
raw["상환구분"] = np.select(conds, values, default_value)

### 시트작성 및 체크

In [37]:
# 최종 시트
최종 = raw.query('진행구분 in ["확정","미확정","실효","완제"]').copy()
확정미확정 = raw.query('진행구분 in ["확정", "미확정"]').copy()

# 전산미등록건 제외
최종.drop(최종[최종.채무자키=='전산미등록건'].index, inplace=True)
확정미확정.drop(확정미확정[확정미확정.채무자키=='전산미등록건'].index, inplace=True)

# 순번열 넣기
raw.insert(0, "순번", range(1,len(raw)+1))
최종.insert(0, "순번", range(1,len(최종)+1))
확정미확정.insert(0, "순번", range(1,len(확정미확정)+1))

In [38]:
# 체크
# 반영OPB
반영OPB오류 = raw[(raw.반영OPB == np.nan)|(raw.반영OPB == 0)]
if len(반영OPB오류) > 0 : display(반영OPB오류)
else : print("반영OPB 오류 없음")

# 확정미확정에 신용회복키 중복여부
신용회복키중복 = 확정미확정[확정미확정.duplicated('신용회복키', keep=False)]
if len(신용회복키중복) > 0 : display(신용회복키중복)
else : print("확정미확정에 신용회복키 중복 없음")

# 채무자키, 계좌키, 신용회복키 매칭 오류
최종키오류 = 최종.query('채무자키=="" or 계좌키=="" or 신용회복키==""')
확정미확정키오류 = 확정미확정.query('채무자키=="" or 계좌키=="" or 신용회복키==""')
print(f"전체 : {len(raw)}, 최종 : {len(최종)}, 확정미확정 : {len(확정미확정)}")
print(f"최종에서 키 없는 행 수 : {len(최종키오류)}, 확정미확정에서 키없 는 행 수 : {len(확정미확정키오류)}")
if len(확정미확정키오류)>0 : display(확정미확정키오류)
else : print("확정미확정 건에 키매칭 실패 없음")

반영OPB 오류 없음


,순번,채무자키,계좌키,보증인키,신용회복키,진행구분,반영OPB,상환구분,신청인명,신청인주민번호,...,체증전구간의종료회차,체증전구간의기준기간,변제금수취계좌,담보권실행유예기간,특별면책,특별상환유예,특례지원,이자율재조정,재난상환유예,거치이자납입유예
1156,276,20419314,200934900,,,미확정,5308104.0,,이상일,620726-1080113,...,0,0,,0,N,,N,N,N,N
9625,8566,20457557,200929511,,,확정,174303.0,개인,이삼수,670409-1906723,...,0,0,,0,N,,N,N,N,N


전체 : 18722, 최종 : 18449, 확정미확정 : 14387
최종에서 키 없는 행 수 : 74, 확정미확정에서 키없 는 행 수 : 2


,순번,채무자키,계좌키,보증인키,신용회복키,진행구분,반영OPB,상환구분,신청인명,신청인주민번호,...,체증전구간의종료회차,체증전구간의기준기간,변제금수취계좌,담보권실행유예기간,특별면책,특별상환유예,특례지원,이자율재조정,재난상환유예,거치이자납입유예
1156,276,20419314,200934900,,,미확정,5308104.0,,이상일,620726-1080113,...,0,0,,0,N,,N,N,N,N
9625,8566,20457557,200929511,,,확정,174303.0,개인,이삼수,670409-1906723,...,0,0,,0,N,,N,N,N,N


### 상환내역
- 기준일이 금욜이 아닌경우 작성해야 함
- 프리의 경우 정확한 상환후잔액은 산정이 불가해서 기준일은 금요일로만 할 것을 권장

In [ ]:
############# 원리금균등 : 조정후합계 - 총납입원금 + 발생이자, 여기서 이자를 정확히 구하기 어려우므로 무조건 금요일을 기준으로 하자 ##################
# 상환내역 파일읽기
# 폴더경로
if company == "솔림" : 
    deposit_base = r"D:\3.자산\신용회복\솔림\1.신용회복\0.입금관련,담당자변경\2023년"
else : 
    deposit_base = r"D:\3.자산\신용회복\대성\신용회복\입금"

# 파일명 리스트 ############################################################################################
deposit_list = ["20231025_신용회복입금_업로드결과", "20231026_신용회복입금_통합_업로드결과"]
###########################################################################################################

# 입금 파일 읽기
deposit_ori = pd.DataFrame(None)
for f in deposit_list : 
    temp = pd.read_excel(join(deposit_base, f+".xlsx"), dtype=deposit_dtype).fillna("")
    deposit_ori = pd.concat([deposit_ori, temp], axis=0)

# 컬럼명 변경
deposit_ori.rename(columns={'분납키':'신용회복키', '입금자':'신청인명','주민번호':'신청인주민번호', 
'입금자구분':'채무구분','입금원금':'납입원금','입금미수이자':'납입이자','입금연체이자':'납입기타채무','입금합계':'납입합계'}, inplace=True)

c:\Users\sl\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [26]:
# 복사본
deposit = deposit_ori.copy()

##### 상환내역 반영 : 기준일 금요일이면 pass

In [ ]:
# 기준키 : 신용회복키
# 적용시트 : 확정미확정 (어차피 완제,실효는 수정할 필요 없고, 수정하기도 복잡하다.완제및실효로 빠졌는지 체크만)
# (check)실효및완제회차(630,666,777,888,999)의 키값이 확정미확정에 없어야 함. 확인 후 해당행 드랍
실효완제회차 = [630,666,777,888,999,63]
실효완제신용회복키 = deposit.query('납입회차.isin(@실효완제회차)').신용회복키.to_list()
실효완제_ind = 확정미확정[확정미확정.신용회복키.isin(실효완제신용회복키)].index
if len(실효완제_ind)>0 : 
    display(확정미확정[확정미확정.신용회복키.isin(실효완제신용회복키)])
    # 있을리가 없는 데 있으면 확인을 먼저 해야겠다. 드랍하거나 원본을 수정하거나..
else : print('실효완제자 이상 없음')

In [64]:
# 실효,완제건 입금 리스트에서 제거
temp1 = len(deposit)
deposit = deposit.drop(deposit.query('납입회차.isin(@실효완제회차)').index).reset_index(drop=True)
temp2 = len(deposit)
print(f'실효완제건수 : {temp1-temp2}')
# 입금을 납입회차 역순정렬 후 신용회복키로 그룹화
deposit.sort_values(['신용회복키','납입회차'], ascending=[True,False], inplace=True)

1306
1306


In [86]:
# 그룹 순회하면서 기준데이터(확정미확정)에 다음을 반영
# 수정할 값 : 납입회차(첫번재 즉 가장 큰 수로 대체), 총납입금액의 합계를 합산, 총납입원금의 합계를 합산, 총납입이자의 합계를 합산, 총납입기타채무의 합계를 합산, 기존 상환후잔액 - 총납입원금의 합계
수정전금액들 = pd.Series([확정미확정.총납입금액.sum(), 확정미확정.총납입원금.sum(), 확정미확정.총납입이자.sum(), 확정미확정.총납입기타채무.sum(), 확정미확정.상환후잔액.sum()])
현재신용회복키 = ""
for i, v in tqdm(deposit.iterrows(), total=len(deposit)) :
    # 기준데이터 수정할 행
    ind = 확정미확정[확정미확정.신용회복키 == v.신용회복키].index
    if 현재신용회복키 != v.신용회복키 : # 신용회복키가 다른 경우만 납입회차 수정
        현재신용회복키 = v.신용회복키
        확정미확정.loc[ind, '납입회차'] = v.납입회차
    
    # 금액은 합산이므로 신용회복키 중복값에 상관없이 계산 적용하면된다
    확정미확정.loc[ind, '총납입금액'] += v.납입합계
    확정미확정.loc[ind, '총납입원금'] += v.납입원금
    확정미확정.loc[ind, '총납입이자'] += v.납입이자
    확정미확정.loc[ind, '총납입기타채무'] += v.납입기타채무
    확정미확정.loc[ind, '상환후잔액'] -= v.납입원금

수정후금액들 = pd.Series([확정미확정.총납입금액.sum(), 확정미확정.총납입원금.sum(), 확정미확정.총납입이자.sum(), 확정미확정.총납입기타채무.sum(), 확정미확정.상환후잔액.sum()])
수정한금액들 = pd.Series([deposit.납입합계.sum(), deposit.납입원금.sum(), deposit.납입이자.sum(), deposit.납입기타채무.sum(), -deposit.납입원금.sum()])

# 총액이 잘 맞는지 확인
print((수정후금액들 == 수정전금액들 + 수정한금액들).unique())

[ True]


### 파일 저장

In [9]:
# 저장
# 기준데이터 저장 경로
filename_write = basedate+"_신용회복_기준데이터_"+company
fullpath_result = join(os.path.dirname(path_base), filename_write+".xlsx")


# 엑셀 파일로 저장
if not os.path.exists(fullpath_result):
    param = {'mode' : 'w'}
else : 
    param = {'mode' : 'a', 'if_sheet_exists' : 'replace'}

with pd.ExcelWriter(fullpath_result, engine='openpyxl', **param) as writer :
    raw.to_excel(writer, sheet_name='원본', index=False)
    최종.to_excel(writer, sheet_name='최종', index=False)
    확정미확정.to_excel(writer, sheet_name='확정,미확정', index=False)